In [ ]:
!pip install LightFM

     |████████████████████████████████| 310 kB 10.4 MB/s 
  Created wheel for LightFM: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=697436 sha256=640c2be5bdb9560d8cd186370cc8944d090b6b63c1d463bc310138680b6450db
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built LightFM


In [ ]:
import pandas as pd
import scipy.sparse
import numpy as np
from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

In [ ]:
users = pd.read_csv('users.csv')
projects = pd.read_csv('projects.csv')

In [ ]:
user_behaviour = pd.merge(users, projects, how='cross')
user_behaviour['user_id'] = np.random.uniform(size=19266)*1000
user_behaviour['user_id'] = user_behaviour['user_id'].astype(int)
user_behaviour[56:76]

,user_id,event_type,project_id,category,main_category
56,318,1,1269204577,3,3
57,496,2,10945100,6,1
58,731,2,1273008525,10,2
59,878,2,1052830150,5,3
60,594,2,1447066521,3,3
61,44,2,1376879513,8,3
62,501,2,11967611,7,1
63,40,2,1534567375,5,3
64,932,2,1516312063,6,1
65,235,2,1537663234,3,3


In [ ]:
print(user_behaviour.shape)

(19266, 5)


In [ ]:
#from pathlib import Path  
#filepath = Path('/content/user_behaviour.csv')  
#user_behaviour.to_csv(filepath)

In [ ]:
user_behaviour.groupby('event_type').size()

event_type
0    11780
1     5320
2     1767
3      399
dtype: int64

In [ ]:
user_behaviour.event_type.unique()

array([1, 2, 0, 3])

In [ ]:
#сортировка и удаление дупликатов
usrBeh = user_behaviour.sort_values(by = 'project_id', ascending=True).drop_duplicates(['project_id'	,'user_id'], keep='last')

In [ ]:
usrArr = usrBeh.user_id.unique()
prdArr = usrBeh.project_id.unique()
print('Кол-во пользователей:', len(usrArr))
print('Кол-во проектов:',len(prdArr))
usrBeh.head()

Кол-во пользователей: 1000
Кол-во проектов: 19


,user_id,event_type,project_id,category,main_category
3211,282,0,10945100,6,1
10887,188,1,10945100,6,1
3648,124,1,10945100,6,1
14459,9,0,10945100,6,1
1121,261,2,10945100,6,1


In [ ]:
print(user_behaviour.shape)

(19266, 5)


In [ ]:
prdArrNp = np.array(prdArr[1], dtype=int)
usrArrNp = np.array(usrArr, dtype=int)
prdArrNp

array(11967611)

In [ ]:
usrBeh2 = pd.pivot_table(usrBeh, values='event_type', index='project_id'	, columns='user_id').fillna(-1)
usrBeh2.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
project_id,,,,,,,,,,,,,,,,,,,,,
10945100,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,...,1.0,0.0,1.0,0.0,-1.0,-1.0,1.0,0.0,0.0,2.0
11967611,1.0,1.0,0.0,1.0,-1.0,-1.0,-1.0,0.0,0.0,2.0,...,-1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0
1030448552,-1.0,0.0,-1.0,-1.0,1.0,0.0,-1.0,2.0,-1.0,-1.0,...,0.0,2.0,0.0,2.0,0.0,-1.0,-1.0,0.0,-1.0,1.0
1049992157,1.0,-1.0,-1.0,1.0,0.0,-1.0,0.0,0.0,2.0,-1.0,...,-1.0,-1.0,1.0,-1.0,0.0,2.0,0.0,-1.0,-1.0,-1.0
1052830150,-1.0,0.0,1.0,-1.0,-1.0,0.0,1.0,0.0,-1.0,-1.0,...,0.0,2.0,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
project_id = list(usrBeh2.index)

In [ ]:
# создание разряженной матрицы
csrUsrBeh = scipy.sparse.csr_matrix(usrBeh, dtype = np.int32)
csrUsrBeh

<12118x5 sparse matrix of type '<class 'numpy.int32'>'
	with 53170 stored elements in Compressed Sparse Row format>

In [ ]:
# предикт будет примерно таким
# usrBeh[100]
#csrUsrBeh

In [ ]:
model = LightFM(no_components=30, loss='warp')
%time model = model.fit(csrUsrBeh, epochs=5, num_threads=4)

CPU times: user 428 ms, sys: 2.63 ms, total: 430 ms
Wall time: 245 ms


In [ ]:
test_precision = precision_at_k(model, csrUsrBeh, k=5).mean()
print(test_precision)

0.8775376


In [ ]:
predictions = model.predict(943, prdArrNp, num_threads=4)